In [ ]:
import ipywidgets
from ipywidgets import Button, Layout, Textarea, HBox, VBox, Label
import os
import glob
import time
import datetime
import subprocess

l = Layout(flex='0 1 auto', height='100px', min_height='100px', width='auto')
process_widget = ipywidgets.Textarea(description='ログ', value='', layout=l)

process_no = 0
def write_log(msg):
    global process_widget, process_no
    process_no = process_no + 1
    process_widget.value = str(process_no) + ": " + msg + "\n" + process_widget.value

In [ ]:
folder_widget = ipywidgets.Dropdown(options=[],description='フォルダ')
dataset_widget = ipywidgets.Dropdown(options=[],description='データ')
zip_button = ipywidgets.Button(description='ZIP')

def change_dataset(change):
    global dataset_widget
    try:
        path = "./" + str(change["new"]) + "/"
        files = os.listdir(path)
        dirs = [f for f in files if os.path.isdir(os.path.join(path, f))]
        dirs = [f for f in dirs if f != ".ipynb_checkpoints"]
        dirs = [f for f in dirs if f != "__pycache__"]
        dirs = [f for f in dirs if f != "model"]
        dirs = [f for f in dirs if f != "model_trt"]
        dirs = sorted(dirs)
        dataset_widget.options = dirs
    except:
        write_log(path + "が存在していません。")
        folder_widget.options = []
        
def change_folder():
    global folder_widget
    try:
        path = "./"
        files = os.listdir(path)
        dirs = [f for f in files if os.path.isdir(os.path.join(path, f))]
        dirs = [f for f in dirs if f != ".ipynb_checkpoints"]
        dirs = [f for f in dirs if f != "__pycache__"]
        dirs = [f for f in dirs if f != "model"]
        dirs = [f for f in dirs if f != "model_trt"]
        dirs = sorted(dirs)
        folder_widget.options = dirs
    except:
        write_log(path + "が存在していません。")
        folder_widget.options = []
folder_widget.observe(change_dataset, names='value')
change_folder()

import subprocess
import os 
import shutil 
import datetime  # 必要な場合、datetimeをインポート

def download_zip(change):
    global folder_widget, dataset_widget

    start_time = datetime.datetime.now()  # 処理開始時刻の取得
    write_log(f"処理開始: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")

    try:
        path = "./zip/"
        if not os.path.exists(path):
            subprocess.call(['mkdir', '-p', path])
            write_log("zipフォルダを生成")

        original_path = "./" + folder_widget.value + "/" + dataset_widget.value
        destination_foldername = f"{folder_widget.value}_{dataset_widget.value}"  # この行を変更
        destination_path = f"./{destination_foldername}"

        # デスティネーションフォルダの作成
        if not os.path.exists(destination_path):
            os.makedirs(destination_path)

        # ファイル・フォルダのコピー
        copy_command = f"cp -r {original_path}/* {destination_path}/"
        os.system(copy_command)

        # ZIP圧縮
        zip_filename = f"{destination_foldername}.zip"
        zip_command = f"zip -r {path}{zip_filename} {destination_path}/*"
        os.system(zip_command)
        write_log(f"保存ファイル名: {path}{zip_filename}")

        # デスティネーションフォルダの削除
        shutil.rmtree(destination_path)
    except Exception as e:
        write_log(f"Error: {str(e)}")

    end_time = datetime.datetime.now()  # 処理終了時刻の取得
    elapsed_time = (end_time - start_time).total_seconds()  # 経過時間の計算
    write_log(f"処理時間: {elapsed_time:.2f} 秒")
    
zip_button.on_click(download_zip)


zip_widget = ipywidgets.VBox([
    folder_widget,
    dataset_widget,
    zip_button,
    process_widget
])
display(zip_widget)